In [132]:
AREA_TYPES = ["desk", "room", "building"]
FEATURES = ["Screen", "Camera", "Microphone"]

USER_COUNT = 500
AREA_COUNT = 20
BOOKING_DURATION_PER_AREA = 60 * 24 * 7

In [133]:
import random
import string
from datetime import datetime, time, date, timedelta

def rand_text(length: int=None, whitespace: bool = False) -> str:
	text = string.ascii_letters
	if whitespace:
		text = text + " "
	if (not length):
		length = random.randint(3, 20)
	text = "".join(random.choices(text, k=length)).capitalize();
	return text

def rand_bool(odds=0.5) -> bool:
	return random.random() < odds

def choices_x(arr: list, num: int = 3) -> list:
	retval = []
	for _ in range(num):
		idx = random.randint(0, len(arr) - 1)
		retval.append(arr.pop(idx))
	return retval

In [134]:
class IdTicker:
	index: int = 0;

	def __init__(self):
		self.index = 0

	def get(self) -> int:
		self.index += 1
		return self.index - 1

In [135]:
user_ticker = IdTicker()
class User:
	id: int = 0
	first_name: str = ""
	last_name: str = ""
	email: str = ""

	def __init__(self):
		self.id = user_ticker.get()
		self.first_name = rand_text()
		self.last_name = rand_text()
		self.email = (rand_text() + "@" + rand_text() + "." + rand_text(random.randint(2,3)))

	def __str__(self):
		return f"{self.first_name} {self.last_name}: {self.email}"

users: dict[int, User] = {}
for i in range(USER_COUNT):
	user = User()
	users[user.id] = user

print(users[1])

Kyvb Qizdjxpwxso: Uaakaki@Seyfdo.Qvi


In [136]:
area_ticker = IdTicker()
class Area:
	id: int = 0
	administrators: list[int] = []
	super_area: int = 0
	area_type: str = "desk"
	capacity: int = 0
	name: str = ""
	description: str = ""
	reservable: bool = False;
	features: list[str] = []

	def __init__(self):
		self.id = area_ticker.get()
		self.administrators = choices_x(list(users.values()))
		if (rand_bool(0.2)):
			self.super_area = random.randint(0, AREA_COUNT - 1)
		self.area_type = random.choice(AREA_TYPES)
		self.capacity = random.randint(0, 10) if rand_bool(0.8) else random.randint(10,100)
		self.name = rand_text()
		self.description = rand_text(random.randint(20, 100), whitespace=True)
		self.reservable = rand_bool(0.9)
		self.features = random.choices(FEATURES, k=random.randint(0, len(FEATURES) - 1))

areas = {}
for _ in range(AREA_COUNT):
	area = Area()
	areas[area.id] = area

In [139]:
booking_ticker = IdTicker()
class Booking:
	id: int = 0
	area: int = 0
	user: int = 0
	start_time: datetime = datetime.now()
	end_time: datetime = datetime.now()
	comment: str = ""

	def __init__(self, area, start, duration):
		self.id = booking_ticker.get()
		self.area = area
		self.user = random.randint(0, USER_COUNT - 1)
		self.start_time = start
		self.end_time = self.start_time + timedelta(minutes=duration)
		self.comment = rand_text(random.randint(20, 100), whitespace=True)

	def __str__(self):
		return f"{self.area}: {self.start_time.time()} - {self.end_time.time()}"

bookings = {}
current_time = datetime.now()
current_time -= timedelta(
	seconds=current_time.second,
	microseconds=current_time.microsecond)
for area in areas:
	current_delta = random.randint(5, 60 * 4)
	while current_delta < BOOKING_DURATION_PER_AREA:
		duration = random.randint(5, 60 * 4)
		start = current_time + timedelta(minutes=current_delta)

		booking = Booking(area, start, duration)
		bookings[booking.id] = booking

		current_delta = current_delta + duration
